In [495]:
import csv 
import re
import pandas as pd
import fitz
import os

# TO DO LIST for clean (each individual)
- Page splits
- Same prof, different locations (multiple new lines)

In [496]:
def extract_text_from_pdf(pdf_path):
    # Open the provided PDF file
    doc = fitz.open(pdf_path)
    text = ""
    
    # Extract text from each page
    for page in doc:
        text += page.get_text()
    
    doc.close()
    return text

In [497]:
# class replacement lines
name_list = [
    'COMPUTER\nSCIENCE\nFOR\nSTEM',
    'INTRODUCTION\nTO\nCOMPUTER\nSCIENCE',
    'DATA\nSTRUCTURES\nAND\nALGORITHMS',
    'FOUNDATIONAL\nPROGRAMMING\nCONCEPTS &\nDESIGN'
    'DATA\nSTRUCTURES\nAND\nALGORITHMS', 
    'PYTHON-BASED\nCOMPUTING:\nBUILDING A\nSENSOR SYSTEM', 
    'INTRODUCTION\nTO THE LINUX\nOPERATING\nSYSTEM', 
    'SOFTWARE\nENGINEERING', 
    'COMPUTER\nORGANIZATION', 
    'DISCRETE\nMATHEMATICS',
    'ADVANCED\nSOFTWARE\nENGINEERING',
    'PRINCIPLES OF\nPROGRAMMING\nLANGUAGES', 
    'DATA BASE\nMANAGEMENT', 
    'ALGORITHMS', 
    'GAME THEORY\nAND NETWORKS',
    'INFORMATION\nSECURITY AND\nPRIVACY', 
    'GRADUATE\nTHESIS /\nDISSERTATION\nRESEARCH\nCREDIT',
    'INTRODUCTION\nTO\nBIOINFORMATICS',
    'COMPOTER\nSCIENCE\nHONORS THESIS',
    'MASTERS\nPROJECT\nCREDITS',
    'HUMAN-ROBOT\nINTERACTION',
    'OPERATING\nSYSTEMS',

    'COMPUTER\nNETWORKS I', 
    'COMPUTER\nNETWORKS II',

    'PROGRAMMING\nCHALLENGES I', 
    'PROGRAMMING\nCHALLENGES II',
    'PROGRAMMING\nCHALLENGES III',

    'FOUNDATIONAL\nPROGRAMMING CONCEPTS &\nDESIGN',
    'CS@MINES\nBRIDGE\nSEMINAR\nCOURSE',
    
    'COMPUTER\nGRAPHICS', 
    'INTRODUCTION\nTO COMPUTER\nVISION',
    'INTRODUCTION\nTO MACHINE\nLEARNING',
    'FORTRAN\nPROGRAMMING',
    'INDUSTRY\nEXPLORATION I', 
    'INDUSTRY\nEXPLORATION II',
    'INDUSTRY\nEXPLORATION III',
    'WEB\nPROGRAMMING', 
    'WEB\nAPPLICATIONS',
    'INTRODUCTION\nTO DATA\nSCIENCE',
    'COMPUTER\nSIMULATION',
    

    'INTRODUCTION\nTO COMPUTER\nSCIENCE - LAB',
    'INTRODUCTION\nTO COMPUTER\nSCIENCE',
    'PROGRAMMING\nCONCEPTS', 
    'DATA\nSTRUCTURES',
    'COMPUTER\nSCIENCE FOR\nSTEM',

    'PYTHON-BASED\nCOMPUTING:\nBUILDING A\nSENSOR\nSYSTEM', 

    'INTRODUCTION\nTO\nCRYPTOGRAPHY',
    'ELEMENTS OF\nGAMES AND\nGAME\nDEVELOPMENT',
    'ELEMENTS OF\nCOMPUTING\nSYSTEMS',
    'COMPILER\nDESIGN',
    'HUMAN-\nCENTERED\nROBOTICS',

    'MOBILE\nAPPLICATION\nDEVELOPMENT',
]

# Extract text from PDFs

In [498]:
# find all files with .pdf extension in directory schedules
pdf_files = [f for f in os.listdir('../schedules') if f.endswith('.pdf')]

In [499]:
# extract text from pdf files
for pdf_path in pdf_files: 
    text = extract_text_from_pdf("../schedules/" + pdf_path)
    # save text
    with open("../schedules/raw_" + pdf_path.replace(".pdf", "") + '.txt', 'w') as f:

        # strip extra spaces from end of lines
        text = re.sub(r'\s+$', '', text, flags=re.MULTILINE)

        # remove header details
        header_end_marker = 'Location Attribute'
        end_column_location = text.index(header_end_marker)
        text = text[end_column_location + len(header_end_marker) + 1:]

        # renmove footer details
        footer_marker = 'New Search\n© 2024 Ellucian Company L.P. and its affiliates.'
        text = text.replace(footer_marker, '')  

        # replace UG schld with UG_Req 
        ug_req_schld_flag = 'UG\nRequired\nCrse for\nSchdlng'
        ta_ra_flag = 'Summer\nGraduate\nTA/RA Fee'

        text = text.replace(ug_req_schld_flag, 'UG_Req\n').replace(ta_ra_flag, 'Summer_Grad_TARA_Fee\n')

        # replace ONLINE NR tag
        text = text.replace('ONLINE NR', 'ONLINE')
        text = text.replace('ONLINE UG_Req\n', 'ONLINE\nUG_Req\n')
        for code in ["NR", "C"]:
            text = text.replace(f"ONLINE\n{code}", f"ONLINE\n\n{code}")
            text = text.replace(f"UG_Req\n{code}", f"UG_Req\n\n{code}")

        # replace TBA with false scheduling information
        text = text.replace('\nTBA','\nAAA\n00:00\nzm-\n00:00\nzm')

        # replace class names
        for name in name_list:
            text = text.replace(name, name.replace('\n', ' '))

        f.write(text)

In [500]:
SEMESTER_ID = 'Fall23'

In [501]:
# open file to read 
with open(f"../schedules/clean_{SEMESTER_ID}.txt", 'r') as f:
    text = f.read()

In [502]:
# replace block endings
pattern = r'([A-Z]{2} \d{3})\n(NR|C)\n(\d{5})'
nr_pattern = r'(NR NR)\n(NR|C)\n(\d{5})'
replacement = r'\1\n\n\2\n\3'

text = re.sub(pattern, replacement, text)
text = re.sub(nr_pattern, replacement, text)

In [503]:
block_list = text.split('\n\n')
print(block_list)

['NR\n82206 CSCI\n128\nA\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n12:00\npm-\n12:50\npm\n63\n59\n4\n0\n0\n0\n0\n0\n0\nChristine\nLiebe (P),\nRobert Haven\nThompson\n08/21-\n12/15\nBB W210', 'C\n82207 CSCI\n128\nB\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n12:00\npm-\n12:50\npm\n72\n72\n0\n0\n0\n0\n0\n0\n0\nVictoria Laine\nMessmore\n(P), Robert\nHaven\nThompson\n08/21-\n12/15\nMZ 326', 'NR\n82220 CSCI\n128\nC\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n12:00\npm-\n12:50\npm\n86\n85\n1\n0\n0\n0\n0\n0\n0\nDrew Robert\nCornmesser\n(P), Robert\nHaven\nThompson\n08/21-\n12/15\nBB W250', 'NR\n82208 CSCI\n128\nD\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n01:00\npm-\n01:50\npm\n72\n65\n7\n0\n0\n0\n0\n0\n0\nVictoria Laine\nMessmore\n(P), Robert\nHaven\nThompson\n08/21-\n12/15\nMZ 326', 'NR\n82209 CSCI\n128\nE\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n01:00\npm-\n01:50\npm\n63\n59\n4\n0\n0\n0\n0\n0\n0\nChristine\nLiebe (P),\nRobert Haven\nThompson\n08/21-\n12/15\nBB W210', 'NR\n82221 CSCI\n128\

In [504]:
split_list = []
for index,block in enumerate(block_list): 
    container = []
    for line in block.split('\n'):
        if line != '':
            container.append(line)
    print(f"LINE #{index}")
    
    if container != []:
        try: 
            # add end flag (for counts)
            if container[-1] != 'UG_Req' and container[-1] != 'Summer_Grad_TARA_Fee' and container[-1] != 'Elective':
                container.append('No_Special_Notes')

            # remove start flag (for counts)
            if container[0] != 'C' and container[0] != 'NR': 
                raise KeyError
            
            # combine credit ranges
            if container[5][-1] == '-':
                container[5] = container[5] + container[6]
                container.remove(container[6])

            # combine dates
            container[-4] = container[-4] + container[-3]
            container.remove(container[-3])

            # combine times
            container[8] = container[8] + container[9] + container[10] + container[11]
            for i in range(11,8,-1):
                container.remove(container[i])

            # combine instructors
            container[18] = " ".join([container[i].replace(" ,", ",") for i in range(18, len(container) - 3)])
            for i in range(len(container) - 4, 18, -1):
                container.remove(container[i])

            # split CRN from Crse #
            l = container[1].split(" ") 
            container[1] = l[0]
            container.insert(2, l[1])

            # add to container
            split_list.append(container)

            print("\t", container)
            print("\n")

        except: 
            print("ERROR")
            print(block)
            print(container)

print(split_list)

LINE #0
	 ['NR', '82206', 'CSCI', '128', 'A', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '12:00pm-12:50pm', '63', '59', '4', '0', '0', '0', '0', '0', '0', 'Christine Liebe (P), Robert Haven Thompson', '08/21-12/15', 'BB W210', 'No_Special_Notes']


LINE #1
	 ['C', '82207', 'CSCI', '128', 'B', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '12:00pm-12:50pm', '72', '72', '0', '0', '0', '0', '0', '0', '0', 'Victoria Laine Messmore (P), Robert Haven Thompson', '08/21-12/15', 'MZ 326', 'No_Special_Notes']


LINE #2
	 ['NR', '82220', 'CSCI', '128', 'C', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '12:00pm-12:50pm', '86', '85', '1', '0', '0', '0', '0', '0', '0', 'Drew Robert Cornmesser (P), Robert Haven Thompson', '08/21-12/15', 'BB W250', 'No_Special_Notes']


LINE #3
	 ['NR', '82208', 'CSCI', '128', 'D', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '01:00pm-01:50pm', '72', '65', '7', '0', '0', '0', '0', '0', '0', 'Victoria Laine Messmore (P), Robert Haven Thompson', '08

In [505]:
# save to csv
with open(f'../schedules/{SEMESTER_ID}.csv', 'w') as f:
    writer = csv.writer(f)

    # write header
    writer.writerow(['Select', 'CRN', 'Subj', 'CrseNum', 'Section', 'CMP', 
                        'Credits', 'Title', 'Days', 'Time', 
                        'Cap', 'Act', 'Rem', 'WL Cap', 'WL Act', 'WL Rem', 'XL Cap', 'XL Act', 'XL Rem', 
                        'Instructors', 'Date', 'Location', 'Attributes'])
    writer.writerows(split_list)

In [506]:
# load csv into df
df = pd.read_csv(f'../schedules/{SEMESTER_ID}.csv')
display(df)
print(df.columns)

,Select,CRN,Subj,CrseNum,Section,CMP,Credits,Title,Days,Time,...,WL Cap,WL Act,WL Rem,XL Cap,XL Act,XL Rem,Instructors,Date,Location,Attributes
0,NR,82206,CSCI,128,A,C,3.000,COMPUTER SCIENCE FOR STEM,MWF,12:00pm-12:50pm,...,0,0,0,0,0,0,"Christine Liebe (P), Robert Haven Thompson",08/21-12/15,BB W210,No_Special_Notes
1,C,82207,CSCI,128,B,C,3.000,COMPUTER SCIENCE FOR STEM,MWF,12:00pm-12:50pm,...,0,0,0,0,0,0,"Victoria Laine Messmore (P), Robert Haven Thom...",08/21-12/15,MZ 326,No_Special_Notes
2,NR,82220,CSCI,128,C,C,3.000,COMPUTER SCIENCE FOR STEM,MWF,12:00pm-12:50pm,...,0,0,0,0,0,0,"Drew Robert Cornmesser (P), Robert Haven Thompson",08/21-12/15,BB W250,No_Special_Notes
3,NR,82208,CSCI,128,D,C,3.000,COMPUTER SCIENCE FOR STEM,MWF,01:00pm-01:50pm,...,0,0,0,0,0,0,"Victoria Laine Messmore (P), Robert Haven Thom...",08/21-12/15,MZ 326,No_Special_Notes
4,NR,82209,CSCI,128,E,C,3.000,COMPUTER SCIENCE FOR STEM,MWF,01:00pm-01:50pm,...,0,0,0,0,0,0,"Christine Liebe (P), Robert Haven Thompson",08/21-12/15,BB W210,No_Special_Notes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NR,82507,CSCI,475,B,C,3.000,INFORMATION SECURITY AND PRIVACY,MW,10:00am-10:50am,...,25,0,25,0,0,0,Guannan Liu (P),08/21-12/15,AH 140,UG_Req
59,C,81704,CSCI,475,LA,C,0.000,INFORMATION SECURITY AND PRIVACY,F,10:00am-10:50am,...,25,0,25,0,0,0,Phillip Richardson Romig (P),08/21-12/15,MZ 022,UG_Req
60,NR,81705,CSCI,475,LB,C,0.000,INFORMATION SECURITY AND PRIVACY,F,01:00pm-01:50pm,...,25,0,25,0,0,0,Guannan Liu (P),08/21-12/15,MZ 026,UG_Req
61,NR,82465,CSCI,475,LC,C,0.000,INFORMATION SECURITY AND PRIVACY,F,10:00am-10:50am,...,0,0,0,0,0,0,Phillip Richardson Romig (P),08/21-12/15,GC PET,UG_Req


Index(['Select', 'CRN', 'Subj', 'CrseNum', 'Section', 'CMP', 'Credits',
       'Title', 'Days', 'Time', 'Cap', 'Act', 'Rem', 'WL Cap', 'WL Act',
       'WL Rem', 'XL Cap', 'XL Act', 'XL Rem', 'Instructors', 'Date',
       'Location', 'Attributes'],
      dtype='object')
